## Preprocessing

In [17]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [18]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
# Determine the number of unique values in each column.
unique_value_counts = application_df.nunique()
print(unique_value_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [20]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [21]:
# Choose a cutoff value
cutoff_value = 500  # You can adjust this cutoff value as needed

# Get the value counts of application types
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Identify application types to be replaced with "Other"
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index

# Replace in the DataFrame
application_df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other", inplace=True)

# Check to make sure binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [22]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [23]:
# You may find it helpful to look at CLASSIFICATION value counts >1

classification_counts=application_df['CLASSIFICATION'].value_counts().loc[lambda x : x >1]

In [24]:
# Choose a cutoff value (e.g., 1000) for classification counts
cutoff_value = 1000

# Get the counts of each classification
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Create a list of classifications to be replaced based on the cutoff value
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [25]:
# Convert categorical data to numeric with `pd.get_dummies`

df = pd.DataFrame(application_df)

# Convert 'Category' column to dummy variables
df = pd.get_dummies(df).astype(int)

In [26]:
# Split our preprocessed data into our features and target arrays
# Assuming you have a DataFrame 'application_df' with preprocessed data

# Define your features (X) and target (y)
X = df.drop(columns=['IS_SUCCESSFUL'],axis=1)
y = df['IS_SUCCESSFUL']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=70)

In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

#### First trail - Adding more hidden layers

In [28]:

nn_model = tf.keras.models.Sequential()
input_dim = X_train.shape[1]
# # Input Layer
nn_model.add(Dense(units=80, activation='relu', input_dim=input_dim))


# First hidden layer 
nn_model.add(Dense(units=30, activation='relu'))

# 2nd hidden layer 
nn_model.add(Dense(units=30, activation='relu'))

# Output layer
nn_model.add(Dense(units=1, activation='sigmoid'))  # For binary classification

# Check the structure of the model
nn_model.summary()

# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train your model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model on the test data
loss, accuracy = nn_model.evaluate(X_test_scaled, y_test)

# Print evaluation results
print(f'Loss: {loss}, Accuracy: {accuracy}')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 dense_9 (Dense)             (None, 80)                3520      
                                                                 
 dense_10 (Dense)            (None, 30)                2430      
                                                                 
 dense_11 (Dense)            (None, 30)                930       
                                                                 
 dense_12 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6911 (27.00 KB)
Trainable params: 6911 (27.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 1s 722us/step - loss: 0.5694 - accuracy: 0.7201
Epoch 2/50
804/804 [==============================] - 1s 733us/step - loss: 0.5542 - accuracy: 0.7302
Epoch 3/50
804/804 [==============================] - 1s 686us/step - loss: 0.5514 - accuracy: 0.7316


#### 2nd trail - Changing number of nuerons

In [29]:
# X = df.drop(columns=['ASK_AMT'])  # Exclude the outliner
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data Preprocessing
# Feature Scaling (Standardization)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Configuration

nn_model2 = tf.keras.models.Sequential()

# Input Layer
input_dim = X_train_scaled.shape[1]
nn_model2.add(Dense(units=64, activation='relu', input_dim=input_dim))

# Hidden Layer
nn_model2.add(Dense(units=64, activation='relu'))

# Output Layer
nn_model2.add(Dense(units=1, activation='sigmoid'))

# Compile the model
# optimizer = Adam(learning_rate=0.001)
nn_model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train your model
fit_model = nn_model2.fit(X_train_scaled, y_train, epochs=50)
# Evaluate the model on the test data
loss, accuracy = nn_model2.evaluate(X_test_scaled, y_test)

# Print evaluation results
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50
858/858 [==============================] - 1s 646us/step - loss: 0.5702 - accuracy: 0.7186
Epoch 2/50
858/858 [==============================] - 1s 621us/step - loss: 0.5547 - accuracy: 0.7267
Epoch 3/50
858/858 [==============================] - 1s 614us/step - loss: 0.5523 - accuracy: 0.7298
Epoch 4/50
858/858 [==============================] - 1s 621us/step - loss: 0.5496 - accuracy: 0.7320
Epoch 5/50
858/858 [==============================] - 1s 631us/step - loss: 0.5498 - accuracy: 0.7321
Epoch 6/50
858/858 [==============================] - 1s 603us/step - loss: 0.5480 - accuracy: 0.7330
Epoch 7/50
858/858 [==============================] - 1s 634us/step - loss: 0.5473 - accuracy: 0.7317
Epoch 8/50
858/858 [==============================] - 1s 615us/step - loss: 0.5467 - accuracy: 0.7343
Epoch 9/50
858/858 [==============================] - 1s 640us/step - loss: 0.5459 - accuracy: 0.7328
Epoch 10/50
858/858 [==============================] - 1s 617us/step - loss: 0.545

Third trail - droping column

In [30]:
X = df.drop(columns=['ASK_AMT'])  # Exclude the outliner
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data Preprocessing
# Feature Scaling (Standardization)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Configuration

nn_model3 = tf.keras.models.Sequential()

# Input Layer
input_dim = X_train_scaled.shape[1]
nn_model3.add(Dense(units=64, activation='relu', input_dim=input_dim))

# Hidden Layer
nn_model3.add(Dense(units=64, activation='relu'))

# Output Layer
nn_model3.add(Dense(units=1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
nn_model3.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train your model
fit_model = nn_model3.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model on the test data
loss, accuracy = nn_model3.evaluate(X_test_scaled, y_test)

# Print evaluation results
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50
858/858 [==============================] - 1s 614us/step - loss: 0.0561 - accuracy: 0.9834
Epoch 2/50
858/858 [==============================] - 1s 606us/step - loss: 0.0039 - accuracy: 0.9994
Epoch 3/50
858/858 [==============================] - 1s 612us/step - loss: 0.0026 - accuracy: 0.9998
Epoch 4/50
858/858 [==============================] - 1s 607us/step - loss: 0.0011 - accuracy: 1.0000
Epoch 5/50
858/858 [==============================] - 1s 616us/step - loss: 3.2107e-05 - accuracy: 1.0000
Epoch 6/50
858/858 [==============================] - 1s 600us/step - loss: 1.6166e-05 - accuracy: 1.0000
Epoch 7/50
858/858 [==============================] - 1s 601us/step - loss: 9.8305e-06 - accuracy: 1.0000
Epoch 8/50
858/858 [==============================] - 1s 599us/step - loss: 6.1457e-06 - accuracy: 1.0000
Epoch 9/50
858/858 [==============================] - 1s 605us/step - loss: 3.7887e-06 - accuracy: 1.0000
Epoch 10/50
858/858 [==============================] - 1s 599u

In [31]:
# Export our model to HDF5 file
nn_model.save("Optimized AlphabetSoupCharity.h5")

c:\Users\Sujatha\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
